# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [ ]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [2]:
import pandas as pd
obj = pd.Series(["Евгения гр.БСТ1903", "Илья БВТ1904", "Анна БСТ 19-04"])
obj

ModuleNotFoundError: No module named 'pandas'

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 5

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import re
import nltk
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    

In [3]:
recipes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/data1/recipes_sample.csv', delimiter=',')
rez = recipes[["id", "minutes"]]
strr = f"|{'id':^13}|{'minutes':^13}|\n|{'':-^27}|\n"
for i in range(5):
  b = rez.sample(replace = False)
  strr = strr + f"|{b['id'].item():^13}|{b['minutes'].item():^13}|\n" 
print(strr)

|     id      |   minutes   |
|---------------------------|
|   436663    |     15      |
|   440156    |     75      |
|   134042    |     30      |
|    94338    |      5      |
|   100156    |     20      |



2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [5]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут'
)

In [4]:
def show_info(name, steps, minutes, author_id):
  rez=f"\"{name.title()}\"\n\n"
  a=1
  for i in steps:
    rez = rez + f"{a}. {i.capitalize()}\n"
    a = a+1
  rez = rez + f"{'':-^10}\nАвтор: {author_id}\nСреднее время приготовления: {minutes} минут"
  return rez


In [11]:
recipes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/data1/recipes_sample.csv', delimiter=',')
with open('/content/drive/MyDrive/Colab Notebooks/data/data1/steps_sample.xml', 'r', encoding="utf-8") as f:
  step = BeautifulSoup(f, 'xml')

In [13]:
r = step.recipes.find_all('recipe')
dictt = {}
for i in r:
    dictt[i.id.text] = [step.text.replace('\n', '') for step in i.steps if step.text!='\n']

#dictt

In [14]:
recipe_170895 = recipes.index[recipes['id']==170895][0]
print(show_info(name=recipes.at[recipe_170895, 'name'], steps = dictt['170895'], minutes = recipes.at[recipe_170895, 'minutes'], author_id=recipes.at[recipe_170895, 'contributor_id']))

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [15]:
for i in dictt['25082']:
  rez = re.findall(r"\d+\s(?:minute|minutes|hour|hours)\b", i) 
  if rez:
    print(rez)

['20 minutes']
['10 minutes']
['2 hours']
['10 minutes']
['20 minutes', '30 minutes']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [16]:
rez=recipes['description'].str.extract(r"^this([\w\s]*),\s?but").any(axis=1)
res=recipes[rez]['description']

In [17]:
print(res.head(3).values)

["this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing."
 "this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers."
 'this is kind of similar to some of the other versions out there, but it is the best and easiest i have found']


In [18]:
print(res.count())

134


5\. В текстах шагов рецептов _обыкновенные дроби_ имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби в обыкновенных дробях. Выведите на экран шаги этого рецепта после их изменения.

In [19]:
for i in dictt['72367']:
  print(re.sub('\s\/\s','/', i))

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [22]:
text = str(dictt.values())
nltk.download('punkt')

reg = r"\w+[a-zA-Z]"
tokenizer = RegexpTokenizer(reg)
word = tokenizer.tokenize(text)

for i in word:
  i = i.lower()
  if not i.isalpha():
    word.remove(i)

rez = list(set(word))
print(len(rez))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


15158


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [23]:
from nltk.tokenize import sent_tokenize
def len_des(text):
  return len(sent_tokenize(str(text)))
  
recipes_sent = recipes['description'].apply(len_des)
recipes.loc[recipes_sent.nlargest(5).index]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN
